In [1]:
!pip install aiostream
!pip install func_adl_uproot

from coffea.processor.servicex import DataSource, Analysis
from coffea.processor.servicex import DaskExecutor, LocalExecutor 
from func_adl import ObjectStream

import pandas as pd
import servicex as sx
from func_adl_servicex import ServiceXSourceUpROOT
from func_adl_uproot import UprootDataset
from servicex import ServiceXDataset

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [2]:
class Processor(Analysis):
    def process(events):
        output = self.accumulator.identity()
        
        dataset = events.metadata["dataset"]
        
        muons = events.Muon
        MET = events.MET.pt
        
        output['cutflow']['all events'] += ak.size(muons, axis=0)
        output['cutflow']['all muons'] += ak.sum(ak.num(muons, axis=1))
        
        # Get all combinations of muon pairs in every event.
        dimuons = ak.combinations(muons, 2, fields=['i0', 'i1'])
        
        # Check that pairs have opposite charge.
        opposites = (dimuons['i0'].charge != dimuons['i1'].charge)
        
        # Get only muons with mass between 60 and 120. Add the dimuon p4's, take mass.
        limits = ((dimuons['i0'] + dimuons['i1']).mass >= 60) & ((dimuons['i0'] + dimuons['i1']).mass < 120)
        
        # Mask the dimuons with the opposites and the limits to get dimuons with opposite charge and mass between 60 and 120 GeV.
        good_dimuons = dimuons[opposites & limits]
        
        # Mask the MET to get it only if an associated dimuon pair meeting the conditions exists.
        good_MET = MET[ak.num(good_dimuons, axis=1) >= 1]
        
        output['cutflow']['final events'] += ak.size(good_MET, axis=0)
        
        output['MET'].fill(dataset=dataset, MET=good_MET)
        
        return output

In [8]:
def get_MET(source: ObjectStream) -> ObjectStream:
    return source.Select(lambda event: event.MET_pt)

ds = ServiceXSourceUpROOT("root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root", 'Events', backend_name='open_uproot')
ds.return_qastle = True
# MET = get_MET(ds)

dataset = [ServiceXDataset("root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root", backend_name='open_uproot')]
datasource = DataSource(query=get_MET(ds), metadata={'dataset': 'Run2012B_SingleMu'}, datasets=dataset)

In [9]:
from dask.distributed import Client

analysis = Processor()
executor = LocalExecutor()
#executor = DaskExecutor(client_addr="tls://matousadamec-40gmail-2ecom.dask.coffea.casa:8786")

async def run_updates_stream(accumulator_stream):
  global first
  count = 0
  async for coffea_info in accumulator_stream:
    count += 1
    print(count, coffea_info)
  return coffea_info
result = await run_updates_stream(executor.execute(analysis, datasource))

root://eospublic.cer...:   0%|          | 0/9000000000.0 [00:00]

        root://eospublic.cer... Downloaded:   0%|          | 0/9000000000.0 [00:00]

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/servicex_%24%7BUSER%7D/data/3aa86067-af4a-4995-9beb-40570dc5e726/root___eospublic.cern.ch__eos_root-eos_benchmark_Run2012B_SingleMu.root.parquet'